In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re
import numpy as np
from PIL import Image
from io import BytesIO
import requests
import os
import json
from urllib.parse import unquote
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
import pandas as pd

In [4]:
data=pd.read_csv("coampanies50_final.csv")
company_list=data['COMPANY_NAME'].to_list()
company_list=company_list[0:200]
company_uids=data['UID'].to_list()
company_uids=company_uids[0:200]

In [5]:
json_list = []

for company,uid in zip(company_list,company_uids):
    retry_count = 0
    
    while retry_count < 3:
        print(uid)
        options = Options()
        options.profile = r"C:\Users\ajai\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default"
        options.binary_location = r"C:\Users\ajai\Desktop\Tor Browser\Browser\firefox.exe"
        driver = webdriver.Firefox(options=options)
        url = 'https://www.goodfirms.co/company/' + str(company)
        print(url)
        driver.get(url)

        try:
            #Read More
            try:
                read_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.read-more-summary')))
                read_more_button.click()
            except (NoSuchElementException, TimeoutException):
                read_more = ''

            #more locations
            try:
                more_locations_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'show-all-location')))
                more_locations_button.click()
            except Exception as e:
                pass

            #name of company
            try:
                company_name = driver.find_element(By.CSS_SELECTOR, "h1[itemprop='name']").text
            except Exception:
                company_name = ''

            locations_dict_list=[]

            try:
                time.sleep(2)
                country_elements = driver.find_elements(By.CSS_SELECTOR, "div.profile-location-country[itemprop='addressCountry']")
                inc=0
                for country_element in country_elements:
                    inc+=1
                    try:
                        country1 = country_element.text
                    except NoSuchElementException:
                        country1 = ""
                    
                    try:
                        address_parent = country_element.find_element(By.XPATH, "./ancestor::div[@class='profile-location']")
                    except NoSuchElementException:
                        address_parent = ""
                    
                    try:
                        street_address_element = address_parent.find_element(By.CSS_SELECTOR, "span[itemprop='streetAddress']")
                        street_address = street_address_element.text
                    except (NoSuchElementException, AttributeError):
                        street_address = ""
                    
                    try:
                        locality_elements = address_parent.find_elements(By.CSS_SELECTOR, "span[itemprop='addressLocality']")
                        localities = []
                        for locality_element in locality_elements:
                            localities.append(locality_element.text)
                    except (NoSuchElementException, AttributeError):
                        locality = ""

                    try:
                        city1=localities[0]
                        state1=localities[1]
                    except IndexError:
                        city1=''
                        state1=''

                    
                    try:
                        postal_code_element = address_parent.find_element(By.CSS_SELECTOR, "span[itemprop='postalCode']")
                        postal_code = postal_code_element.text.strip()
                    except (NoSuchElementException, AttributeError):
                        postal_code=''

                    
                    if country1=='India' and postal_code!='':
                        endpoint=f'https://api.postalpincode.in/pincode/{postal_code}'
                        response=requests.get(endpoint)
                        pincode_info=json.loads(response.text)
                        area=pincode_info[0]['PostOffice'][0]['Name'].strip()
                        city=pincode_info[0]['PostOffice'][0]['Region'].strip()
                        state=pincode_info[0]['PostOffice'][0]['State'].strip()
                        country=pincode_info[0]['PostOffice'][0]['Country'].strip()
                        locations_dict={
                            'country':country,
                            'state':state,
                            'city':city,
                            'area':area
                        }
                    else:
                        locations_dict={
                            'country':country1,
                            'state':state1,
                            'city':city1,
                            'area':street_address,  
                        }
                    if inc==1:
                        headquarters_dict=locations_dict
                    else:
                        locations_dict_list.append(locations_dict)
            except Exception as e:
                locations_dict_list=''
                headquarters_dict=''

            #website link
            try:
                website_url=driver.find_element(By.CSS_SELECTOR,'a.visit-website').get_attribute('href')
            except NoSuchElementException:
                website_url=''

            #profile pic
            try:
                profile_src=driver.find_element(By.CSS_SELECTOR,'.profile-logo img').get_attribute('data-src')
                response = requests.get(profile_src)
                profile_pic=f'gallery//images//{uid}//profile_pic//{company}_portfolio_pic.png'
                profile_pic1=profile_pic.replace("//",'/')
                directory = os.path.dirname(profile_pic)
                if not os.path.exists(directory):
                    os.makedirs(directory)
                with open(profile_pic, "wb") as f:
                    f.write(response.content)
            except NoSuchElementException:
                profile_pic=''


            #profile Tagline
            try:
                profile_tagline=driver.find_element(By.CSS_SELECTOR,'p.profile-tagline').text.strip()
            except NoSuchElementException:
                profile_tagline=''

            #Review Rating
            try:
                rating=driver.find_element(By.CSS_SELECTOR,'.review-rating').text.strip()
            except NoSuchElementException:
                rating=''

            #Profile Overview Text
            try:
                profile_overview=driver.find_element(By.CSS_SELECTOR,'div.profile-overview').text.strip().replace("\n","")
                # print(profile_overview)
            except NoSuchElementException:
                profile_overview=''

            #profile-pricing custom_tooltip
            try:
                profile_pricing_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-pricing').text.strip()
                profile_pricing_custom_tooltip=profile_pricing_custom_tooltip.replace('$', '').replace('/hr', '')
            except NoSuchElementException:
                profile_pricing_custom_tooltip=''

            #profile employees custom tooltip
            try:
                profile_employees_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-employees').text.strip()
            except NoSuchElementException:
                profile_employees_custom_tooltip=''

            #profile-founded custom_tooltip
            try:
                profile_founded_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-founded').text.strip()
            except NoSuchElementException:
                profile_founded_custom_tooltip=''

            #Social media
            try:
                social_media=[k.get_attribute('href') for k in driver.find_elements(By.CSS_SELECTOR,'.social-profile a')]

                pattern_twiiter = re.compile(r'.*twitter.*')
                try:
                    twitter = next((url for url in social_media if pattern_twiiter.search(url)), None).replace("+"," ").strip()
                except AttributeError:
                    twitter=''
                    pass
                pattern_facebook = re.compile(r'.*facebook.*')
                try:
                    facebook = next((url for url in social_media if pattern_facebook.search(url)), None).replace("+"," ").strip()
                except AttributeError:
                    facebook=''
                    pass
                pattern_linkedin = re.compile(r'.*linkedin.*')
                try:
                    linkedin = next((url for url in social_media if pattern_linkedin.search(url)), None).replace("+"," ").strip()
                except AttributeError:
                    linkedin=''
                    pass
                pattern_yt = re.compile(r'.*youtube.*')
                try:
                    youtube = next((url for url in social_media if pattern_yt.search(url)), None).replace("+"," ").strip()
                except AttributeError:
                    youtube=''
                    pass
                pattern_ig = re.compile(r'.*instagram.*')
                try:
                    instagram = next((url for url in social_media if pattern_ig.search(url)), None).replace("+"," ").strip()
                except AttributeError:
                    instagram=''
                    pass
            except NoSuchElementException:
                twiiter=''
                facebook=''
                linkedin=''
                youtube=''
                instagram=''
                pass    
            
            #Service Focus
            try:
                parent_service=driver.find_element(By.CSS_SELECTOR,'.focus-container.service-focus-container')
                service_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_service.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                service_percent=[k.text for k in parent_service.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                # print(service_name)
                # print(service_percent)
                service_dict_list=[]
                for y,z in zip(service_name,service_percent):
                    service_dict={
                        'service-type':y.strip(),
                        'service-percent':z.strip()
                    }
                    service_dict_list.append(service_dict)
            except NoSuchElementException:
                service_dict_list=''

            #Client Focus
            try:
                parent_client=driver.find_element(By.CSS_SELECTOR,'.client-focus-container')
                client_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_client.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                client_percent=[k.text for k in parent_client.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                client_dict_list=[]
                for y,z in zip(client_name,client_percent):
                    client_dict={
                        'client-type':y.strip(),
                        'client-percent':z.strip()
                    }
                    client_dict_list.append(client_dict)
            except NoSuchElementException:
                client_dict_list=''


            #industry Focus
            try:
                parent_industry=driver.find_element(By.CSS_SELECTOR,'.industry-focus-container')
                industry_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_industry.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                industry_percent=[k.text for k in parent_industry.find_elements(By.CLASS_NAME,"entity-chart-bar")]
                industry_dict_list=[]
                for y,z in zip(industry_name,industry_percent):
                    industry_dict={
                        'industry-type':y.strip(),
                        'industry-percent':z.strip()
                    }
                    industry_dict_list.append(industry_dict)
            except NoSuchElementException:
                industry_dict_list=''

            #Key Clients
            try:
                key_client_u1=driver.find_element(By.CSS_SELECTOR,'.key-client-list-section ul')
                key_client_list=[k.text.strip() for k in key_client_u1.find_elements(By.TAG_NAME,'li')]
            except NoSuchElementException:
                key_client_list=''

            try:
                full_address=', '.join(list(headquarters_dict.values()))
            except Exception as e:
                full_address=''

            try:
                country=list(headquarters_dict.values())[0].strip()
                state=list(headquarters_dict.values())[1].strip()
                city=list(headquarters_dict.values())[2].strip()
                area=list(headquarters_dict.values())[3].strip()
            except Exception:
                area=''
                city=''
                state=''
                country=''


            new_data={
            'uid':uid,
            'url':url,
            'website-url':website_url,
            'title':company_name,
            'profile-pic':profile_pic1,
            'profile-slogan':profile_tagline,
            'star-rating':rating,
            'profile-overview':profile_overview,
            'full-address':full_address,
            'headquarters':full_address,
            "country":country,
            "state":state,
            "city":city,
            'area':area,
            'other-locations':locations_dict_list, 
            "social-media-facebook": facebook,
            "social-media-twitter": twitter,
            "social-media-linkedin": linkedin,
            "social-media-instagram": instagram,
            "social-media-youtube": youtube,
            'no-of-employees':profile_employees_custom_tooltip,
            'average-hourly-rate':profile_pricing_custom_tooltip,
            'year-founded':profile_founded_custom_tooltip,
            'services-focus':service_dict_list,
            'client-focus':client_dict_list,
            'industry-focus':industry_dict_list,
            'clients':key_client_list
            }
            print(new_data)
            json_list.append(new_data)
            
            break  # Break out of the while loop if no exception occurs

        except Exception as e:
            retry_count += 1
            print(f"Exception occurred: {e}. Retrying... Attempt {retry_count}")

            if retry_count >= 3:
                raise RuntimeError(f"Failed to process company {company} after 3 retries.") 

            # Wait for a few seconds before retrying
            time.sleep(2)
        finally:
            driver.quit()

json_data = json.dumps(json_list, indent=4)
print(json_data)


7770000201
https://www.goodfirms.co/company/the-go-to-guy
{'uid': 7770000201, 'url': 'https://www.goodfirms.co/company/the-go-to-guy', 'website-url': 'http://www.thegotoguy.co/', 'title': 'The Go-To Guy', 'profile-pic': 'gallery/images/7770000201/profile_pic/the-go-to-guy_portfolio_pic.png', 'profile-slogan': 'Helping Build Brands', 'star-rating': '', 'profile-overview': 'The Go-To Guy! is a full-service creative and digital agency based out of India & Dubai (UAE). Consultants have made Strategy - Overrated, Marketing – A complicated myth and Branding – An untouchable convict. We, at The Go-To Guy! are attempting to break this norm and surprise the world every day!We are "Your Idea Execution Agency". If you have an idea, we will take it to the world. We build brand identities, activate presence & engage consumers - Online, Social and mobile.Recognition:Award-Winning Company in India for its Branding, Digital Strategy, and SMM services by TechBehemoths 2023Dubai\'s Most-Reviewed B2B Par

In [4]:
with open(r"gf_30mar/Goodfirms JSON 49 companies.json", "w") as json_file:
    json_file.write(json_data)

In [2]:
s="Binmile is a Product Engineering & Custom Software Development Company with offices across the globe.We are an enterprise software engineering company that helps organizations implement their digital journeys at speed. An ISO 9001:2008 & 27001:2013 certified company, strengthened by our unique Global Delivery Model.From software development to platform modernization and product engineering to R&D, we offer a full suite of services for companies across different industries. Binmile provides expert consultancy in the development, customization, and integration of complex enterprise-level product development - SaaS, IoT solutions, test automation, and advanced web & mobile applications. \nCertifications"

In [5]:
s

'Binmile is a Product Engineering & Custom Software Development Company with offices across the globe.We are an enterprise software engineering company that helps organizations implement their digital journeys at speed. An ISO 9001:2008 & 27001:2013 certified company, strengthened by our unique Global Delivery Model.From software development to platform modernization and product engineering to R&D, we offer a full suite of services for companies across different industries. Binmile provides expert consultancy in the development, customization, and integration of complex enterprise-level product development - SaaS, IoT solutions, test automation, and advanced web & mobile applications. \nCertifications'

In [4]:
s.replace("\n","")

'Binmile is a Product Engineering & Custom Software Development Company with offices across the globe.We are an enterprise software engineering company that helps organizations implement their digital journeys at speed. An ISO 9001:2008 & 27001:2013 certified company, strengthened by our unique Global Delivery Model.From software development to platform modernization and product engineering to R&D, we offer a full suite of services for companies across different industries. Binmile provides expert consultancy in the development, customization, and integration of complex enterprise-level product development - SaaS, IoT solutions, test automation, and advanced web & mobile applications. Certifications'

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re
import numpy as np
from PIL import Image
from io import BytesIO
import requests
import os
import json
from urllib.parse import unquote
import undetected_chromedriver as uc
from selenium_stealth import stealth
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.firefox.options import Options

In [3]:
data=pd.read_csv("Sample Company Names.csv")
company_list=data['COMPANY_NAME'].to_list()
company_list=company_list[0:20]

In [4]:
json_list=[]
uid=7770000000
for company in company_list:
    uid+=1
    options = Options()
    options.add_argument('--no-sandbox')
    options.profile = r"C:\Users\ajai\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default"
    options.binary_location = r"C:\Users\ajai\Desktop\Tor Browser\Browser\firefox.exe"
    driver = webdriver.Firefox(options=options)
    url='https://www.goodfirms.co/company/'+str(company)
    print(url)
    driver.get(url)

    #Read More
    try:
        read_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.read-more-summary')))
        read_more_button.click()
    except (NoSuchElementException,TimeoutException):
        read_more=''

    #more locations
    try:
        more_locations_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'show-all-location')))
        more_locations_button.click()
    except (NoSuchElementException,TimeoutException):
        pass

    #name of company
    try:
        company_name=driver.find_element(By.CSS_SELECTOR,"h1[itemprop='name']").text
    except Exception:
        company_name=''
    
    locations_dict_list=[]

    try:
        time.sleep(2)
        country_elements = driver.find_elements(By.CSS_SELECTOR, "div.profile-location-country[itemprop='addressCountry']")
        inc=0
        for country_element in country_elements:
            inc+=1
            try:
                country1 = country_element.text
            except NoSuchElementException:
                country1 = ""
            
            try:
                address_parent = country_element.find_element(By.XPATH, "./ancestor::div[@class='profile-location']")
            except NoSuchElementException:
                address_parent = ""
            
            try:
                street_address_element = address_parent.find_element(By.CSS_SELECTOR, "span[itemprop='streetAddress']")
                street_address = street_address_element.text
            except (NoSuchElementException, AttributeError):
                street_address = ""
            
            try:
                locality_elements = address_parent.find_elements(By.CSS_SELECTOR, "span[itemprop='addressLocality']")
                localities = []
                for locality_element in locality_elements:
                    localities.append(locality_element.text)
            except (NoSuchElementException, AttributeError):
                locality = ""

            try:
                city1=localities[0]
                state1=localities[1]
            except IndexError:
                city1=''
                state1=''

            
            try:
                postal_code_element = address_parent.find_element(By.CSS_SELECTOR, "span[itemprop='postalCode']")
                postal_code = postal_code_element.text.strip()
            except (NoSuchElementException, AttributeError):
                postal_code=''

            
            if country1=='India' and postal_code!='':
                endpoint=f'https://api.postalpincode.in/pincode/{postal_code}'
                response=requests.get(endpoint)
                pincode_info=json.loads(response.text)
                area=pincode_info[0]['PostOffice'][0]['Name'].strip()
                city=pincode_info[0]['PostOffice'][0]['Region'].strip()
                state=pincode_info[0]['PostOffice'][0]['State'].strip()
                country=pincode_info[0]['PostOffice'][0]['Country'].strip()
                locations_dict={
                    'country':country,
                    'state':state,
                    'city':city,
                    'area':area
                }
            else:
                locations_dict={
                    'country':country1,
                    'state':state1,
                    'city':city1,
                    'area':street_address,  
                }
            if inc==1:
                headquarters_dict=locations_dict
            else:
                locations_dict_list.append(locations_dict)
    except Exception as e:
        locations_dict_list=''
        headquarters_dict=''

    #website link
    try:
        website_url=driver.find_element(By.CSS_SELECTOR,'a.visit-website').get_attribute('href')
    except NoSuchElementException:
        website_url=''

    #profile pic
    try:
        profile_src=driver.find_element(By.CSS_SELECTOR,'.profile-logo img').get_attribute('data-src')
        response = requests.get(profile_src)
        profile_pic=f'gallery//images//{uid}//profile_pic//{company}_portfolio_pic.png'
        profile_pic1=profile_pic.replace("//",'/')
        directory = os.path.dirname(profile_pic)
        if not os.path.exists(directory):
            os.makedirs(directory)
        with open(profile_pic, "wb") as f:
            f.write(response.content)
    except NoSuchElementException:
        profile_pic=''


    #profile Tagline
    try:
        profile_tagline=driver.find_element(By.CSS_SELECTOR,'p.profile-tagline').text.strip()
    except NoSuchElementException:
        profile_tagline=''

    #Review Rating
    try:
        rating=driver.find_element(By.CSS_SELECTOR,'.review-rating').text.strip()
    except NoSuchElementException:
        rating=''

    #Profile Overview Text
    try:
        profile_overview=driver.find_element(By.CSS_SELECTOR,'div.profile-overview').text.strip()
        # print(profile_overview)
    except NoSuchElementException:
        profile_overview=''

    #profile-pricing custom_tooltip
    try:
        profile_pricing_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-pricing').text.strip()
        profile_pricing_custom_tooltip=profile_pricing_custom_tooltip.replace('$', '').replace('/hr', '')
    except NoSuchElementException:
        profile_pricing_custom_tooltip=''

    #profile employees custom tooltip
    try:
        profile_employees_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-employees').text.strip()
    except NoSuchElementException:
        profile_employees_custom_tooltip=''

    #profile-founded custom_tooltip
    try:
        profile_founded_custom_tooltip=driver.find_element(By.CLASS_NAME,'profile-founded').text.strip()
    except NoSuchElementException:
        profile_founded_custom_tooltip=''

    #Social media
    try:
        social_media=[k.get_attribute('href') for k in driver.find_elements(By.CSS_SELECTOR,'.social-profile a')]

        pattern_twiiter = re.compile(r'.*twitter.*')
        try:
            twitter = next((url for url in social_media if pattern_twiiter.search(url)), None).replace("+"," ").strip()
        except AttributeError:
            twitter=''
            pass
        pattern_facebook = re.compile(r'.*facebook.*')
        try:
            facebook = next((url for url in social_media if pattern_facebook.search(url)), None).replace("+"," ").strip()
        except AttributeError:
            facebook=''
            pass
        pattern_linkedin = re.compile(r'.*linkedin.*')
        try:
            linkedin = next((url for url in social_media if pattern_linkedin.search(url)), None).replace("+"," ").strip()
        except AttributeError:
            linkedin=''
            pass
        pattern_yt = re.compile(r'.*youtube.*')
        try:
            youtube = next((url for url in social_media if pattern_yt.search(url)), None).replace("+"," ").strip()
        except AttributeError:
            youtube=''
            pass
        pattern_ig = re.compile(r'.*instagram.*')
        try:
            instagram = next((url for url in social_media if pattern_ig.search(url)), None).replace("+"," ").strip()
        except AttributeError:
            instagram=''
            pass
    except NoSuchElementException:
        twiiter=''
        facebook=''
        linkedin=''
        youtube=''
        instagram=''
        pass    
    
    #Service Focus
    try:
        parent_service=driver.find_element(By.CSS_SELECTOR,'.focus-container.service-focus-container')
        service_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_service.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        service_percent=[k.text for k in parent_service.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        # print(service_name)
        # print(service_percent)
        service_dict_list=[]
        for y,z in zip(service_name,service_percent):
            service_dict={
                'service name':y.strip(),
                'service percent':z.strip()
            }
            service_dict_list.append(service_dict)
    except NoSuchElementException:
        service_dict_list=''

    #Client Focus
    try:
        parent_client=driver.find_element(By.CSS_SELECTOR,'.client-focus-container')
        client_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_client.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        client_percent=[k.text for k in parent_client.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        client_dict_list=[]
        for y,z in zip(client_name,client_percent):
            client_dict={
                'client name':y.strip(),
                'client percent':z.strip()
            }
            client_dict_list.append(client_dict)
    except NoSuchElementException:
        client_dict_list=''


    #industry Focus
    try:
        parent_industry=driver.find_element(By.CSS_SELECTOR,'.industry-focus-container')
        industry_name=[k.get_attribute('data-content').replace("<i>",'').replace("</i>",'') for k in parent_industry.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        industry_percent=[k.text for k in parent_industry.find_elements(By.CLASS_NAME,"entity-chart-bar")]
        industry_dict_list=[]
        for y,z in zip(industry_name,industry_percent):
            industry_dict={
                'industry name':y.strip(),
                'industry percent':z.strip()
            }
            industry_dict_list.append(industry_dict)
    except NoSuchElementException:
        industry_dict_list=''

    #Key Clients
    try:
        key_client_u1=driver.find_element(By.CSS_SELECTOR,'.key-client-list-section ul')
        key_client_list=[k.text.strip() for k in key_client_u1.find_elements(By.TAG_NAME,'li')]
    except NoSuchElementException:
        key_client_list=''

    try:
        full_address=', '.join(list(headquarters_dict.values()))
    except Exception as e:
        full_address=''

    try:
        area=list(headquarters_dict.values())[0].strip()
        city=list(headquarters_dict.values())[1].strip()
        state=list(headquarters_dict.values())[2].strip()
        country=list(headquarters_dict.values())[3].strip()
    except Exception:
        area=''
        city=''
        state=''
        country=''


    new_data={
    'uid':uid,
    'url':url,
    'website url':website_url,
    'title':company_name,
    'profile pic':profile_pic1,
    'profile slogan':profile_tagline,
    'star rating':rating,
    'profile overview':profile_overview,
    'full address':full_address,
    'headquarters':full_address,
    "country":country,
    "state":state,
    "city":city,
    'area':area,
    'other locations':locations_dict_list, 
    "Social-Media-Facebook": facebook,
    "Social-Media-Twitter": twitter,
    "Social-Media-Linkedin": linkedin,
    "Social-Media-Instagram": instagram,
    "Social-Media-Youtube": youtube,
    'No of Employees':profile_employees_custom_tooltip,
    'Average Hourly Rate':profile_pricing_custom_tooltip,
    'Year Founded':profile_founded_custom_tooltip,
    'services focus':service_dict_list,
    'client focus':client_dict_list,
    'industry focus':industry_dict_list,
    'clients':key_client_list
    }
    print(new_data)
    json_list.append(new_data)
    driver.quit()

json_data = json.dumps(json_list, indent=4)
print(json_data)
    

https://www.goodfirms.co/company/alliancetek
{'uid': 7770000001, 'url': 'https://www.goodfirms.co/company/alliancetek', 'website url': 'https://www.alliancetek.com/?utm_source=goodfirms&utm_medium=main&utm_campaign=goodfirmsprofile', 'title': 'AllianceTek', 'profile pic': 'gallery/images/7770000001/profile_pic/alliancetek_portfolio_pic.png', 'profile slogan': 'Empowering IT Solutions', 'star rating': '5.0', 'profile overview': 'With a talented team of 200+ software developers, project managers, and designers, we excel in delivering end-to-end tailored solutions empowered with the latest tools, technology stack, and methods. At AllianceTek, we don’t just design and develop your project. Rather, we audit and engineer the entire project development process. We work with you to strategize and find the best solutions specific to your requirements while regularly taking feedback throughout the process.\nOur clients do not have to worry about culture gaps or communication problems that can ar

ElementClickInterceptedException: Message: Element <button class="show-all-location" type="button"> is not clickable at point (1074,408) because another element <div id="profileCta" class="call-to-action-slide show"> obscures it
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
ElementClickInterceptedError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:331:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:162:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:121:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:214:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:97:31


In [6]:
profile_pricing_custom_tooltip

'< $25/hr'

In [8]:
profile_pricing_custom_tooltip.replace(r'\$|/hr',"")

'< $25/hr'

In [9]:
ajai=''

In [10]:
ajai.replace('','')

''